## 복습 프로세스
- 네이버 뉴스 검색 제목 크롤링
- 네이버 뉴스 크롤링
- 네이버 자동 검색
- 네이버 연관 검색어 3차 크롤링(엑셀)
- 유튜브 랭킹 크롤링
- 판다스 이름 분석
- 선별진료소 지도

### 준비

In [1]:
# 라이브러리
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MiniMap

# chromedriver 버전 업데이트
chromedriver_autoinstaller.install()

'C:\\Users\\Drimsys\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\96\\chromedriver.exe'

### 네이버 뉴스 검색 제목 크롤링

In [2]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "http://www.naver.com"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

In [3]:
# 테스트
title=soup.select("a.news_tit")
len(title)

0

In [4]:
# 검색 리스트
search_word_list=["파이썬", "자바", "카프카"]

# 크롤링 진행
# 검색 리스트 반복
for search_word in search_word_list:
    # 검색 리스트 페이지 열기
    url=f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={search_word}"
    browser.get(url)
    
    # 찍기
    html=browser.page_source
    soup=BeautifulSoup(html, "html.parser")
    
    time.sleep(1)
    
    # 제목 태그
    title=soup.select("a.news_tit")[0].text
    print(title.strip())

이패스비즈, 파이썬 활용 주가예측 분석툴 제작 특강 진행
구글, 취약한 로그4j 사용하는 자바 패키지 35863개 찾아
“브렉시트 뒤의 추악한 분열정치, ‘바퀴벌레 총리’로 풍자”


### 네이버 뉴스 크롤링

In [5]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=031&aid=0000637022"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

In [6]:
# 테스트
title=soup.select("h3#articleTitle")
date=soup.select("span.t11")
press=soup.select("div.press_logo > a > img")
len(press)

1

In [7]:
# 속성값 테스트
soup.select("div.press_logo > a > img")[0]["title"]

'아이뉴스24'

In [8]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=031&aid=0000637022"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 태그
title=soup.select("h3#articleTitle")[0].text.strip()
date=soup.select("span.t11")[0].text.strip()
press=soup.select("div.press_logo > a > img")[0]["title"].strip()

# 출력
print(title)
print(date)
print(press)

승현준 삼성리서치 소장 "개방성과 협업이 소프트웨어의 핵심"
2021.11.18. 오전 10:01
아이뉴스24


### 네이버 자동 검색

In [9]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

word_list=["파이썬", "자바"]
for word in word_list:
    # 검색 단어 입력
    searching=browser.find_elements_by_css_selector("input#query")[0]
    searching.clear()
    searching.send_keys(word)

    # 검색
    search_btn=browser.find_elements_by_css_selector("button#search_btn")[0]
    search_btn.click()
    
    # 되돌아가기
    browser.back()

<ipython-input-9-23316ca8094c>:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]
<ipython-input-9-23316ca8094c>:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  search_btn=browser.find_elements_by_css_selector("button#search_btn")[0]


### 네이버 연관 검색어 3차 크롤링(엑셀)

In [10]:
# excel 파일 열기
raw=pd.read_excel("./data/searching_word.xlsx")
word_list=raw["검색어"]
word_list

0    파이썬
1     자바
Name: 검색어, dtype: object

In [11]:
# 날짜 테스트
now=datetime.now().isoformat()[:10]
now

'2021-12-22'

In [12]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

# 검색 단어 입력
searching=browser.find_elements_by_css_selector("input#query")[0]
searching.clear()
searching.send_keys(word_list[0])

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 테스트
related_word=soup.select("ul.kwd_lst._kwd_list > li.item._item > a > span.fix")
len(related_word)

<ipython-input-12-f041bcb89542>:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]


10

In [13]:
# excel 파일로 저장 함수
def save_excel(result):
    df=pd.DataFrame(results)
    df.columns=["검색어", "연관검색어", "검색일시"]
    df.to_excel("./result/Result_Searching_1.xlsx", index=False)
    return df

In [15]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

word=word_list[0]
results=[]

# 검색 단어 입력
searching=browser.find_elements_by_css_selector("input#query")[0]
searching.clear()
searching.send_keys(word_list[0])

# 검색 시간
now=datetime.now().isoformat()[:10]

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 연관 검색어 태그
for tag in soup.select("ul.kwd_lst._kwd_list > li.item._item > a > span.fix"):
    related_word=tag.text
    if related_word != word:
        results.append([word, related_word, now])
df=save_excel(results)
df.head()

<ipython-input-15-a41eca74fb2f>:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]


,검색어,연관검색어,검색일시
0,파이썬,파이썬 자격증,2021-12-22
1,파이썬,파이썬 독학,2021-12-22
2,파이썬,파이썬 리스트,2021-12-22
3,파이썬,파이썬 설치,2021-12-22
4,파이썬,파이썬 함수,2021-12-22


### 유튜브 랭킹 크롤링

In [16]:
# 브라우저 열기
browser = webdriver.Chrome() # 컴퓨터용 브라우저

url="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1"
browser.get(url)

time.sleep(1)

# 100개 채널, 채널명, 카테고리, 구독자수, 조회수, 영상 개수
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

### 판다스 이름 분석

In [17]:
raw=pd.read_csv("./data/babyNamesUS.csv")
raw.head()

,StateCode,Sex,YearOfBirth,Name,Number
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7


### 선별진료소 지도

In [18]:
# 테스트
# 서울역 위도, 경도
seoul_station = [37.5536387,126.9671714]
map=folium.Map(location=seoul_station, zoom_start=12)

# 마커하기
folium.Marker(seoul_station, tooltip="Seoul", popup="Station").add_to(map)
map